In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


np.random.seed(42)

# Number of samples
num_normal_samples = 50000  
num_anomalous_samples = 2500  


normal_heart_rate = np.random.normal(loc=75, scale=5, size=num_normal_samples).astype(int)
normal_spo2 = np.random.normal(loc=98, scale=1, size=num_normal_samples)  


normal_heart_rate = np.clip(normal_heart_rate, 60, 100) 
normal_spo2 = np.clip(normal_spo2, 95, 100)


anomalous_heart_rate = np.random.normal(loc=130, scale=10, size=num_anomalous_samples).astype(int)  
anomalous_spo2 = np.random.normal(loc=80, scale=5, size=num_anomalous_samples)  

# Clip to ensure values are outside healthy ranges
anomalous_heart_rate = np.clip(anomalous_heart_rate, 110, 150)  
anomalous_spo2 = np.clip(anomalous_spo2, 70, 90)  


heart_rate = np.concatenate([normal_heart_rate, anomalous_heart_rate])
spo2 = np.concatenate([normal_spo2, anomalous_spo2])
labels = np.concatenate([np.ones(num_normal_samples), -np.ones(num_anomalous_samples)]) 


df = pd.DataFrame({
    'Heart Rate (BPM)': heart_rate,
    'SpO2': spo2,
    'Label': labels
})


df = df.sample(frac=1, random_state=42).reset_index(drop=True)





train_df = df[df['Label'] == 1]  
test_df = df  


X_train = train_df[['Heart Rate (BPM)', 'SpO2']]  
X_test = test_df[['Heart Rate (BPM)', 'SpO2']] 
y_test = test_df['Label'] 


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


clf = IsolationForest(n_estimators=200, max_samples=256, contamination=0.01, random_state=42)
clf.fit(X_train_scaled) 


test_df['Predicted Label'] = clf.predict(X_test_scaled) 
test_df['Anomaly Score'] = clf.decision_function(X_test_scaled) 




In [12]:
print("\nClassification Report:")
print(classification_report(y_test, test_df['Predicted Label'], target_names=["Normal", "Anomaly"]))


Classification Report:
              precision    recall  f1-score   support

      Normal       0.83      1.00      0.91      2500
     Anomaly       1.00      0.99      0.99     50000

    accuracy                           0.99     52500
   macro avg       0.92      0.99      0.95     52500
weighted avg       0.99      0.99      0.99     52500



In [ ]:
from flask import Flask, request, jsonify
import numpy as np


app = Flask(__name__)


@app.route('/predict', methods=['POST'])
def predict():
    try:
    
        data = request.json
        heart_rate = data.get("heart_rate")
        spo2 = data.get("spo2")

   
        if heart_rate is None or spo2 is None:
            return jsonify({"error": "Missing heart_rate or spo2 value"}), 400

      
        input_data = np.array([[heart_rate, spo2]])
        input_scaled = scaler.transform(input_data)  

       
        prediction = clf.predict(input_scaled)[0]
        anomaly_score = clf.decision_function(input_scaled)[0]

       
        result = "Normal" if prediction == 1 else "Anomalous"

        return jsonify({
            "heart_rate": heart_rate,
            "spo2": spo2,
            "prediction": result,
            "anomaly_score": anomaly_score
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500



if __name__ == '__main__':
   app.run(port=5001)
